## Problem Statement 

You need to build a model that is able to classify customer complaints based on the products/services. By doing so, you can segregate these tickets into their relevant categories and, therefore, help in the quick resolution of the issue.

You will be doing topic modelling on the <b>.json</b> data provided by the company. Since this data is not labelled, you need to apply NMF to analyse patterns and classify tickets into the following five clusters based on their products/services:

* Credit card / Prepaid card

* Bank account services

* Theft/Dispute reporting

* Mortgages/loans

* Others 


With the help of topic modelling, you will be able to map each ticket onto its respective department/category. You can then use this data to train any supervised model such as logistic regression, decision tree or random forest. Using this trained model, you can classify any new customer complaint support ticket into its relevant department.

## Pipelines that needs to be performed:

You need to perform the following eight major tasks to complete the assignment:

1.  Data loading

2. Text preprocessing

3. Exploratory data analysis (EDA)

4. Feature extraction

5. Topic modelling 

6. Model building using supervised learning

7. Model training and evaluation

8. Model inference

## Importing the necessary libraries

In [ ]:
import json 
import numpy as np
import pandas as pd
import re, nltk, spacy, string
import en_core_web_sm
nlp = en_core_web_sm.load()
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from plotly.offline import plot
import plotly.graph_objects as go
import plotly.express as px

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from pprint import pprint

## Loading the data

The data is in JSON format and we need to convert it to a dataframe.

In [ ]:
# Opening JSON file 
f = f = open("data/complaints-2021-05-14_08_16.json")# Write the path to your data file and load it 
  
# returns JSON object as  
# a dictionary 
data = json.load(f)
df=pd.json_normalize(data)

## Data preparation

In [ ]:
# Inspect the dataframe to understand the given data.

df.head()

In [ ]:
# Inspect the dataset information
df.info()

In [ ]:
#print the column names
df.columns

In [ ]:
df.shape

In [ ]:
#Assign new column names
#Removing _source from column name
df_columns = df.columns
new_column_names = [ column.replace("_source.","") for column in df_columns]
df.columns = new_column_names
df.info()

In [ ]:
#Checking the null values 
df.isnull().sum()

In [ ]:
print("Number of empty string in complaint_what_happend :",(df["complaint_what_happened"]=='').sum())

In [ ]:
#Assign nan in place of blanks in the complaints column
df["complaint_what_happened"] = df["complaint_what_happened"].apply(lambda x : pd.NA if len(x)==0 else x)
print("Number of empty string in complaint :",(df["complaint_what_happened"]=='').sum())

In [ ]:
# Checking nan values in complaint column
print("Now nan values in complain column : ",df["complaint_what_happened"].isnull().sum())
df["complaint_what_happened"].isna().sum()

In [ ]:
#Remove all rows where complaints column is nan
df=df[~df["complaint_what_happened"].isna()]
print("Now nan values in complain column : ",df["complaint_what_happened"].isna().sum())

In [ ]:
df.shape

## Prepare the text for topic modeling

Once you have removed all the blank complaints, you need to:

* Make the text lowercase
* Remove text in square brackets
* Remove punctuation
* Remove words containing numbers


Once you have done these cleaning operations you need to perform the following:
* Lemmatize the texts
* Extract the POS tags of the lemmatized text and remove all the words which have tags other than NN[tag == "NN"].


In [ ]:
# Write your function here to clean the text and remove all the unnecessary elements.
def clean_text(text) :
  text = text.lower() # Make the text lowercase
  text = re.sub(r'\[.*?\]', '', text) # Remove text in square brackets
  text = re.sub(r'[^\w\s]', '', text) # Remove punctuation
  text = re.sub(r'\b\w*\d\w*\b', '', text)  #Remove words containing numbers
  return text;

In [ ]:
df_clean = pd.DataFrame(df['complaint_what_happened'].apply(clean_text))

In [ ]:
df_clean

In [ ]:
#Write your function to Lemmatize the texts
def lemmmatize_text(text):
    sent = []
    doc = nlp(text)
    for token in doc:
        sent.append(token.lemma_)
    return " ".join(sent)

In [ ]:
#Create a dataframe('df_clean') that will have only the complaints and the lemmatized complaints 
df_clean['complaint_lemmatized'] = df_clean['complaint_what_happened'].apply(lemmmatize_text)

In [ ]:
df_clean

In [ ]:
# Import Textblob for extracting noun phrases
!pip install textblob
from textblob import TextBlob

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

In [ ]:
#Write your function to extract the POS tags 

def pos_tag(text):
  # write your code here
  sent = []
  blob = TextBlob(text)
  sent = [word for (word,tag) in blob.tags if tag=='NN']
  return " ".join(sent)

In [ ]:
df_clean["complaint_POS_removed"] =   df_clean['complaint_lemmatized'].apply(pos_tag) #this column should contain lemmatized text with all the words removed which have tags other than NN[tag == "NN"]

In [ ]:
#The clean dataframe should now contain the raw complaint, lemmatized complaint and the complaint after removing POS tags.
df_clean

## Exploratory data analysis to get familiar with the data.

Write the code in this task to perform the following:

*   Visualise the data according to the 'Complaint' character length
*   Using a word cloud find the top 40 words by frequency among all the articles after processing the text
*   Find the top unigrams,bigrams and trigrams by frequency among all the complaints after processing the text. ‘




In [ ]:
# Write your code here to visualise the data according to the 'Complaint' character length

character_length = [len(each_sent) for each_sent in df_clean['complaint_POS_removed']]

sns.displot(character_length, kind='hist', bins=70)
plt.xlabel("Complaint character length")
plt.ylabel("Number of Complaints")
plt.title("Distribution of Complaint character length")
plt.show()

#### Find the top 40 words by frequency among all the articles after processing the text.

In [ ]:
# Installing wordcloud
!pip install wordcloud

from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

In [ ]:
#Using a word cloud find the top 40 words by frequency among all the articles after processing the text

stopwords = set(STOPWORDS)
word_cloud = WordCloud(max_font_size=70, max_words=40, background_color="white", random_state=100, stopwords=stopwords)
word_cloud.generate(str(df_clean['complaint_POS_removed']))
plt.figure(figsize=[15,15])
plt.imshow(word_cloud, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
#Removing -PRON- from the text corpus 
df_clean['Complaint_clean'] = df_clean['complaint_POS_removed'].str.replace('-PRON-', '')

In [ ]:
df_clean.shape

In [ ]:
df_clean

#### Find the top unigrams,bigrams and trigrams by frequency among all the complaints after processing the text.

In [ ]:
# Creating a function to extract top ngrams(unigram/bigram/trigram) based on the function inputs
def get_ngrams(text, n=None, ngram=(1,1)):
  vec = CountVectorizer(stop_words='english', ngram_range=ngram).fit(text)
  bagofwords = vec.transform(text)
  sum_words = bagofwords.sum(axis=0)
  words_frequency = [(word, sum_words[0, index]) for word, index in vec.vocabulary_.items()]
  words_frequency = sorted(words_frequency, key = lambda x: x[1], reverse=True)
  return words_frequency[:n]

In [ ]:
#Write your code here to find the top 30 unigram frequency among the complaints in the cleaned datafram(df_clean). 
top_30words = get_ngrams(df_clean['Complaint_clean'].values.astype('U'), n=30, ngram=(1,1))
df_unigram = pd.DataFrame(top_30words, columns=['unigram', 'count'])
df_unigram

In [ ]:
# Plotting top 30 Unigrams
plt.figure(figsize=[25,6])
sns.barplot(x=df_unigram['unigram'], y=df_unigram['count'])
plt.xticks(rotation=45)
plt.xlabel("Unigram")
plt.ylabel("Count")
plt.title("Count of top 30 Unigrams")
plt.show()

In [ ]:
#Print the top 10 words in the unigram frequency
df_unigram.head(10)

In [ ]:
# Plotting top 10 words
plt.figure(figsize=[25,6])
sns.barplot(x=df_unigram['unigram'].head(10), y=df_unigram['count'])
plt.xticks(rotation=45)
plt.xlabel("Unigram")
plt.ylabel("Count")
plt.title("Count of top 30 Unigrams")
plt.show()

In [ ]:
#Write your code here to find the top 30 bigram frequency among the complaints in the cleaned datafram(df_clean). 
top_30words = get_ngrams(df_clean['Complaint_clean'].values.astype('U'), n=30, ngram=(2,2))
df_bigram = pd.DataFrame(top_30words, columns=['bigram', 'count'])
df_bigram

In [ ]:
# Plotting top 30 Bigrams
plt.figure(figsize=[25,6])
sns.barplot(x=df_bigram['bigram'], y=df_bigram['count'])
plt.xticks(rotation=45)
plt.xlabel("Bigram")
plt.ylabel("Count")
plt.title("Count of top 30 Bigrams")
plt.show()

In [ ]:
#Print the top 10 words in the bigram frequency
df_bigram.head(10)

In [ ]:
# Plotting top 10 Bugrams
plt.figure(figsize=[25,6])
sns.barplot(x=df_bigram['bigram'].head(10), y=df_bigram['count'])
plt.xticks(rotation=45)
plt.xlabel("Bigram")
plt.ylabel("Count")
plt.title("Count of top 10 Bigrams")
plt.show()

In [ ]:
#Write your code here to find the top 30 trigram frequency among the complaints in the cleaned datafram(df_clean). 
top_30words = get_ngrams(df_clean['Complaint_clean'].values.astype('U'), n=30, ngram=(3,3))
df_trigram = pd.DataFrame(top_30words, columns=['trigram', 'count'])
df_trigram

In [ ]:
# Plotting top 30 Trigrams
plt.figure(figsize=[25,6])
sns.barplot(x=df_trigram['trigram'], y=df_trigram['count'])
plt.xticks(rotation=45)
plt.xlabel("Trigram")
plt.ylabel("Count")
plt.title("Count of top 30 Trigrams")
plt.show()

In [ ]:
#Print the top 10 words in the trigram frequency
df_trigram.head(10)

In [ ]:
# Plotting top 10 Trigrams
plt.figure(figsize=[25,6])
sns.barplot(x=df_trigram['trigram'].head(10), y=df_trigram['count'])
plt.xticks(rotation=45)
plt.xlabel("Trigram")
plt.ylabel("Count")
plt.title("Count of top 10 Trigrams")
plt.show()

## The personal details of customer has been masked in the dataset with xxxx. Let's remove the masked text as this will be of no use for our analysis

In [ ]:
df_clean['Complaint_clean'] = df_clean['Complaint_clean'].str.replace('xxxx','')

In [ ]:
df_clean.shape

In [ ]:
#All masked texts has been removed
df_clean

## Feature Extraction
Convert the raw texts to a matrix of TF-IDF features

**max_df** is used for removing terms that appear too frequently, also known as "corpus-specific stop words"
max_df = 0.95 means "ignore terms that appear in more than 95% of the complaints"

**min_df** is used for removing terms that appear too infrequently
min_df = 2 means "ignore terms that appear in less than 2 complaints"

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer

In [ ]:
#Write your code here to initialise the TfidfVectorizer 

tfidf = TfidfVectorizer(min_df=2, max_df=0.95, stop_words='english')

#### Create a document term matrix using fit_transform

The contents of a document term matrix are tuples of (complaint_id,token_id) tf-idf score:
The tuples that are not there have a tf-idf score of 0

In [ ]:
#Write your code here to create the Document Term Matrix by transforming the complaints column present in df_clean.
documet_term_matrix = tfidf.fit_transform(df_clean['Complaint_clean'])

In [ ]:
documet_term_matrix

## Topic Modelling using NMF

Non-Negative Matrix Factorization (NMF) is an unsupervised technique so there are no labeling of topics that the model will be trained on. The way it works is that, NMF decomposes (or factorizes) high-dimensional vectors into a lower-dimensional representation. These lower-dimensional vectors are non-negative which also means their coefficients are non-negative.

In this task you have to perform the following:

* Find the best number of clusters 
* Apply the best number to create word clusters
* Inspect & validate the correction of each cluster wrt the complaints 
* Correct the labels if needed 
* Map the clusters to topics/cluster names

In [ ]:
from sklearn.decomposition import NMF

## Manual Topic Modeling
You need to do take the trial & error approach to find the best num of topics for your NMF model.

The only parameter that is required is the number of components i.e. the number of topics we want. This is the most crucial step in the whole topic modeling process and will greatly affect how good your final topics are.

In [ ]:
#Load your nmf_model with the n_components i.e 5
num_topics = 5 

#keep the random_state =40
nmf_model = NMF(n_components=num_topics, random_state=40)

In [ ]:
nmf_model.fit(documet_term_matrix)
len(tfidf.get_feature_names_out())

In [ ]:
#Print the Top15 words for each of the topics
H = nmf_model.components_       # Topic-term matrix
words = np.array(tfidf.get_feature_names_out())
topic_words = pd.DataFrame(np.zeros((num_topics, 15)), index=[f'Topic {i + 1}' for i in range(num_topics)],
                           columns=[f'Word {i + 1}' for i in range(15)]).astype(str)
for i in range(num_topics):
    ix = H[i].argsort()[::-1][:15]
    topic_words.iloc[i] = words[ix]

topic_words

**Observation** Looking at the topics above, for each topic, we can give a label based on their products/services:

Topic 1 = Bank account services

Topic 2 = Credit card / Prepaid card

Topic 3 = Others

Topic 4 = Theft/Dispute reporting

Topic 5 = Mortgages/loans

In [ ]:
#Create the best topic for each complaint in terms of integer value 0,1,2,3 & 4
topic_results = nmf_model.transform(documet_term_matrix)

In [ ]:
#Assign the best topic to each of the cmplaints in Topic Column
df_clean['Topic'] =  topic_results.argmax(axis=1)#write your code to assign topics to each rows.

In [ ]:
df_clean.shape

In [ ]:
df_clean.head()

In [ ]:
#Print the first 5 Complaint for each of the Topics
df_clean_first_five=df_clean.groupby('Topic').head(5)
df_clean_first_five.sort_values('Topic')

In [ ]:
df_clean.shape

#### After evaluating the mapping, if the topics assigned are correct then assign these names to the relevant topic:
* Bank Account services
* Credit card or prepaid card
* Theft/Dispute Reporting
* Mortgage/Loan
* Others

In [ ]:
#Create the dictionary of Topic names and Topics

topic_names = { 0:"Bank account services", 1:"Credit card / Prepaid card", 2:"Others",3:"Theft/Dispute reporting", 4:"Mortgages/loans"   }
#Replace Topics with Topic Names
df_clean['Topic'] = df_clean['Topic'].map(topic_names)

In [ ]:
df_clean.shape

In [ ]:
df_clean

## Supervised model to predict any new complaints to the relevant Topics.

You have now build the model to create the topics for each complaints.Now in the below section you will use them to classify any new complaints.

Since you will be using supervised learning technique we have to convert the topic names to numbers(numpy arrays only understand numbers)

In [ ]:
#Create the dictionary again of Topic names and Topics

topic_names = {  "Bank account services":0, "Credit card / Prepaid card":1, "Others":2, "Theft/Dispute reporting":3, "Mortgages/loans":4  }
#Replace Topics with Topic Names
df_clean['Topic'] = df_clean['Topic'].map(topic_names)

In [ ]:
df_clean.shape

In [ ]:
df_clean.head()

In [ ]:
#Keep the columns"complaint_what_happened" & "Topic" only in the new dataframe --> training_data
training_data=df_clean[['complaint_what_happened', 'Topic']]

In [ ]:
training_data.shape

In [ ]:
training_data

####Apply the supervised models on the training data created. In this process, you have to do the following:
* Create the vector counts using Count Vectoriser
* Transform the word vecotr to tf-idf
* Create the train & test data using the train_test_split on the tf-idf & topics


In [ ]:
# Import pickle to save and load the model
#import pickle

In [ ]:
#Write your code to get the Vector count
count_vectorizer = CountVectorizer()
X_train_cout = count_vectorizer.fit_transform(training_data['complaint_what_happened'])

In [ ]:
#Write your code here to transform the word vector to tf-idf
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_cout)

You have to try atleast 3 models on the train & test data from these options:
* Logistic regression
* Decision Tree
* Random Forest
* Naive Bayes (optional)

**Using the required evaluation metrics judge the tried models and select the ones performing the best**

In [ ]:
# Write your code here to build any 3 models and evaluate them using the required metrics

# Logistic regression : 

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [ ]:
# Split data into train and test
X_train, X_test, y_train, y_test = train_test_split(X_train_tfidf, training_data['Topic'], test_size=0.3, random_state=42)

In [ ]:
X_train

In [ ]:
X_test

In [ ]:
logistic_regression = LogisticRegression().fit(X_train, y_train)
predicted = logistic_regression.predict(X_test)

topicnames_target = ["Bank account services", "Credit card / Prepaid card", "Others", "Theft/Dispute reporting", "Mortgages/loans"]
print(classification_report(y_true=y_test, y_pred=predicted,target_names = topicnames_target))

# Decision Tree 

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
# Decision tree classifier
decision_tree_classifier = DecisionTreeClassifier().fit(X_train, y_train)
predicted = decision_tree_classifier.predict(X_test)

print(classification_report(y_pred=predicted, y_true=y_test, target_names = topicnames_target))

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
random_forest_classifier = RandomForestClassifier(max_depth=10)
random_forest_classifier.fit(X_train, y_train)
predicted = random_forest_classifier.predict(X_test)

print(classification_report(y_pred=predicted, y_true=y_test,target_names = topicnames_target))

# Naive Bayes (optional)

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
gaussian_navie_bayes = GaussianNB().fit(X_train.toarray(), y_train)
predicted = gaussian_navie_bayes.predict(X_test.toarray())

print(classification_report(y_pred=predicted, y_true=y_test,target_names = topicnames_target))

# Observations :

1. Logistic regression : Accuracy is 0.92  , f1-scores are 0.93, 0.93, 0.91 ,0.90 and 0.93 for classess "Bank account services", "Credit card / Prepaid card", "Others", "Theft/Dispute reporting", "Mortgages/loans" respectively.

2. Decision Tree : Accuracy is 0.79  , f1-scores are 0.80, 0.81, 0.75, 0.74 and 0.82 for classess "Bank account services", "Credit card / Prepaid card", "Others", "Theft/Dispute reporting", "Mortgages/loans" respectively.

3. Random Forest : Accuracy is 0.72  , f1-scores are 0.79, 0.73, 0.06, 0.73 and 0.79  for classess "Bank account services", "Credit card / Prepaid card", "Others", "Theft/Dispute reporting", "Mortgages/loans" respectively.

4. Naive Bayes (optional) : Accuracy is 0.35  , f1-scores are 0.33,0.39, 0.23, 0.38 and 0.47 for classess "Bank account services", "Credit card / Prepaid card", "Others", "Theft/Dispute reporting", "Mortgages/loans" respectively.


From above analysis , we can see that Logistic regression is giving best accuracy and f1-scores.

# Using Logistic regression to predict ( some examples )

In [ ]:
df_complaints = pd.DataFrame({'complaints': ["I can not get from chase who services my mortgage, who owns it and who has original loan docs", 
                                  "The bill amount of my credit card was debited twice. Please look into the matter and resolve at the earliest.",
                                  "I want to open a salary account at your downtown branch. Please provide me the procedure.",
                                  "Yesterday, I received a fraudulent email regarding renewal of my services.",
                                  "What is the procedure to know my CIBIL score?",
                                  "I need to know the number of bank branches and their locations in the city of Dubai"]})
df_complaints

In [ ]:
def predict_using_linear_regression(text):
    topic_names = {0:'Account Services', 1:'Others', 2:'Mortgage/Loan', 3:'Credit card or prepaid card', 4:'Theft/Dispute Reporting'}
    X_new_counts = count_vectorizer.transform(text)
    X_new_tfidf = tfidf_transformer.transform(X_new_counts)
    predicted = logistic_regression.predict(X_new_tfidf)
    return topic_names[predicted[0]]

In [ ]:
df_complaints['tag'] = df_complaints['complaints'].apply(lambda x: predict_using_linear_regression([x]))
df_complaints